# Hands-on ML - Chapter 3
# Exercise 3-4: Spam Classifier

In [14]:
import os 
import tarfile
import urllib

In [15]:
DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"

In [16]:
HAM_URL = DOWNLOAD_ROOT + '20030228_easy_ham.tar.bz2'
SPAM_URL = DOWNLOAD_ROOT + '20030228_spam.tar.bz2'

In [17]:
SPAM_PATH = os.path.join(os.getcwd(), 'spam')

In [18]:
def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    """Fetches Ham/Spam email data from Apache's servers"""
    
    # If the spam_path directory does not exist, make one
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    
    # Create a new file for each URL specified in the list of tuples
    for filename, url in (('ham.tar.bz2', HAM_URL), 
                         ('spam.tar.bz2', SPAM_URL)):
        # Remember the filepath for the downloaded path
        path = os.path.join(spam_path, filename)
        
        # If the file has not already been downloaded, do so now
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        
        # Extract each file
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=SPAM_PATH)
        tar_bz2_file.close()

In [19]:
fetch_spam_data()

## Loading Data

In [20]:
# Filepaths for directories with spam and ham email files
HAM_DIR = os.path.join(SPAM_PATH, 'easy_ham')
SPAM_DIR = os.path.join(SPAM_PATH, 'spam')

In [21]:
# List of all ham emails - files with at least 20 chars in name are emails
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]

In [24]:
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [25]:
# How many of each kind of email
print("Total non-spam (ham) emails: ", len(ham_filenames))
print("Total spam emails: ", len(spam_filenames))

Total non-spam (ham) emails:  2500
Total spam emails:  500


## Parsing Emails
Can use Python's `email` module to parse emails to handle headers, encoding body content, etc.

In [26]:
import email
import email.policy

In [28]:
def load_email(is_spam, filename, spam_path=SPAM_PATH):
    """Uses `email` module to parse specified email as a stream of bytes"""
    # Load from a different directory based on the argument
    directory = 'spam' if is_spam else 'easy_ham' 
    
    # Open each email file as a stream of bytes to be read 
    with open(os.path.join(spam_path, directory, filename), 'rb') as f:
        # Read each file as a stream of bytes using Python's email parser module
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [30]:
# Read all ham (non-spam) emails
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]

In [31]:
# Read all spam emails
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

## Example of Ham/Spam Emails

In [32]:
# Strip whitespace and header/to/from data from a sample
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [33]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


### Multipart Emails
Also need to deal with cases where emails will have multiple parts (e.g. images, attachments, quoted text, etc.).

In [38]:
def get_email_structure(email):
    """Function to find the different types of emails in the list"""
    # If a single email, just return
    if isinstance(email, str):
        return email
    
    # Otherwise, extract the email's content
    payload = email.get_payload()
    
    # If the payload is a list of smaller units (attachments, images, forwarded emails, quoted emails)
    if isinstance(payload, list):
        # Return a string of all subcomponents concatenated
        return 'multipart({})'.format(",".join([
            get_email_structure(sub_email) for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [39]:
from collections import Counter

In [40]:
def structures_counter(emails):
    """Return object with all distinct types of emails and their counts"""
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

Use these functions to find the most commonly occurring types of ham emails.

In [41]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain,application/pgp-signature)', 66),
 ('multipart(text/plain,text/html)', 8),
 ('multipart(text/plain,text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain,application/octet-stream)', 2),
 ('multipart(text/plain,text/enriched)', 1),
 ('multipart(text/plain,application/ms-tnef,text/plain)', 1),
 ('multipart(multipart(text/plain,text/plain,text/plain),application/pgp-signature)',
  1),
 ('multipart(text/plain,video/mng)', 1),
 ('multipart(text/plain,multipart(text/plain))', 1),
 ('multipart(text/plain,application/x-pkcs7-signature)', 1),
 ('multipart(text/plain,multipart(text/plain,text/plain),text/rfc822-headers)',
  1),
 ('multipart(text/plain,multipart(text/plain,text/plain),multipart(multipart(text/plain,application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain,application/x-java-applet)', 1)]

Do the same for the spam emails.

In [42]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain,text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain,image/jpeg)', 3),
 ('multipart(text/html,application/octet-stream)', 2),
 ('multipart(text/plain,application/octet-stream)', 1),
 ('multipart(text/html,text/plain)', 1),
 ('multipart(multipart(text/html),application/octet-stream,image/jpeg)', 1),
 ('multipart(multipart(text/plain,text/html),image/gif)', 1),
 ('multipart/alternative', 1)]

Most ham emails are plain text, while a substantial number of spam emails are HTML. Most ham emails have a PGP signature, while spam emails don't have any PGP signature.

## Email Headers

In [43]:
for header, value in spam_emails[0].items():
    print(header, ":", value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [44]:
# Focusing specifically on the subject 
spam_emails[0]['Subject']

'Life Insurance - Why Pay More?'

## Train-Test Split

In [45]:
import numpy as np
from sklearn.model_selection import train_test_split

In [47]:
# Concatenate features and labels for ham and spam emails in sequence
X = np.array(ham_emails + spam_emails)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preprocessing - Regular Expressions
- Get rid of all `<head>` elements.
- Converts all `<a>` tags to the word `HYPERLINK`.
- Remove all HTML tags.
- Replace multiple newlines with single newlines for readability.
- Unescape HTML special characters such as `&nbsp;`.

In [48]:
import re
from html import unescape

In [50]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [53]:
# Get all spam emails with only text/html in their content
html_spam_emails = [email for email in X_train[y_train==1] if get_email_structure(email) == 'text/html']

In [54]:
sample_html_spam = html_spam_emails[7]

# Print the first 1000 characters of the spam email
print(sample_html_spam.get_content().strip())[:1000]

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

TypeError: 'NoneType' object is not subscriptable

In [55]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], '...')


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

### Email to Text Function
Writing a more generic function that will convert emails to text, regardless of their content type. Will use the `html_to_plain_text` function defined earlier, but only for a specific subtype of emails.

In [56]:
def email_to_text(email):
    html = None
    
    # Step through all parts of the mail content
    for part in email.walk():
        # Try and get the content type
        ctype = part.get_content_type()
        
        # If not already a text/plain or text/html email
        if not ctype in ('text/plain', 'text/html'):
            continue # Do nothing for now
        # Try to get its content though
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
            
        # If plain, return content
        if ctype == 'text/plain':
            return content
        # Otherwise, html is the same as content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [57]:
# Print the first 100 charactes in the processed email
print(email_to_text(sample_html_spam)[:100], '...')


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Wat ...


## Stemming
The process of replacing words with their root words. This helps keep the vocabulary size low while also preserving the content of the text.

In [60]:
try:
    import nltk
    stemmer = nltk.PorterStemmer()
    for word in ('Computations', 'Computation', 'Computing', 'Computed', 'Compute', 'Compulsive'):
        print(word, "=>", stemmer.stem(word))
except ImportError:
    print("Error: stemming requires NLTK module.")
    stemmer = None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


## Replacing URLs with "URL"

In [61]:
try:
    import urlextract
    
    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print("Error: replacng URLs requires the urlexdtract module.")
    url_extractor = None

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


## Defining Preprocessing Transformer

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin

In [65]:
class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        """Initialize the transformer with boolean instance variables for various preprocessing requirements"""
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
        
    def fit(self, X, y=None):
        # Nothing to change
        return self
    
    def transform(self, X, y=None):
        """Based on each member variable, change the email"""
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [66]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)

In [67]:
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

## Transformer - Word Counts to Vectors
Fit method will build an ordered vocabulary or list of most common words.

Transform will use the vocabulary to convert word counts to vectors.

In [68]:
from scipy.sparse import csr_matrix

In [70]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [71]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)

In [72]:
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)

In [73]:
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [74]:
X_few_vectors.toarray()

array([[ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [99, 11,  9,  8,  3,  1,  3,  1,  3,  2,  3],
       [67,  0,  1,  2,  3,  4,  1,  2,  0,  1,  0]], dtype=int64)

In [76]:
vocab_transformer.vocabulary_

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'url': 5,
 'all': 6,
 'in': 7,
 'christian': 8,
 'on': 9,
 'by': 10}

## Pipeline

In [77]:
from sklearn.pipeline import Pipeline

In [80]:
preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

## Classifier - Logistic Regression

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [82]:
log_clf = LogisticRegression(solver='lbfgs', random_state=42)

In [83]:
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/saadmashkoor/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] .................................. , score=0.98125, total=   0.1s
[CV]  ................................................................
[CV] .................................. , score=0.98375, total=   0.1s
[CV]  ................................................................
[CV] ................................... , score=0.9925, total=   0.1s


/Users/saadmashkoor/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
/Users/saadmashkoor/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s finished


In [86]:
score.mean()

0.9858333333333333

## Precision and Recall

In [87]:
from sklearn.metrics import precision_score, recall_score

In [88]:
X_test_transformed = preprocess_pipeline.transform(X_test)

In [89]:
log_clf = LogisticRegression(solver='lbfgs', random_state=42)

In [90]:
log_clf.fit(X_train_transformed, y_train)

/Users/saadmashkoor/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [91]:
y_pred = log_clf.predict(X_test_transformed)

In [92]:
print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))

Precision: 95.88%


In [93]:
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))

Recall: 97.89%


This is a very good classifier!